# Using a generator function to slice pandas DataFrames and train a model a batch at a time

We're going to train an LSTM to generate 'tweets' (160 character snippets), using a training dataset of nearly a million tweets. As we need to create muliple training examples for each tweet (e.g., training the LSTM to predict each next character based on a 40 character window), this produces 40 training examples for each tweet. It turns out that when expanded out and one-hot encoded on a character level, a mere 3,000 tweets expands to a (120000, 40, 93) matrix, which even represented as bool dtypes in a NumPy ndarray. (While storing data as bool types, rather than int, in C would save quite a bit of space, in Python both bool and int types take a whopping 28 bytes to store. When you multiply it out, this is nearly 12GiB of data (although NumPy gets it down to 4GiB so it is managing to store it more efficiently behind the scenes than we'd be able to do in Python alone). However, even a 4GB chunk of RAM, (with peak memory usage needed to build the array higher still), this starts to cause significant problems on my laptop with 16GiB of RAM, and limits me to around 4,000 tweets before the Jupyter kernel crashes.

Sure, there's some easy wins (training it on an AWS supercomputer, say, or use something slightly less memory-hungry than a Jupyter notebook) but since we'd like to train the model on a data set around 150 - 300 times the size of this, this memory usage is unacceptable, even training on an AWS instance, and ideally we'd like to code in a way that makes the best use of the resources that we have. Let's try coding up the function that loads the data as a generator function and feeding it to the model using the keras fit_generator function rather than holding it all in memory as a massive NumPy array and feeding it in one go.

Let's also use some memory and CPU profiling tools to benchmark each way of doing it, to see where the trade-offs are.

Let's see what efficiency improvements we can make using the tools that we have, before we start thinking about bringing in the big guns.

First, let's testing the logic of a generator function for getting batches of training examples

In [1]:
def batch_generator(data_list, batch_size=8):
    batch = 0
    n_batches = int(len(data_list) / batch_size)

    while True:

        yield data_list[(batch*batch_size):(batch+1)*batch_size]
        
        batch += 1
        batch = batch % n_batches # loop indefinately

In [2]:
my_data = range(32)

b = batch_generator(my_data, batch_size=8)

In [3]:
for i in range(5):
    print(list(next(b)))

    # note that just calling (b) rather than next(b) causes
    #an infinate loop in the batch_generator (while True)

[0, 1, 2, 3, 4, 5, 6, 7]
[8, 9, 10, 11, 12, 13, 14, 15]
[16, 17, 18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 2, 3, 4, 5, 6, 7]


In [4]:
next(b)

range(8, 16)

Great! that's exactly what we need. Now let's code up the `convert_tweet_to_xy` function into a generator function version, using this model.

In [32]:
# now contained in the utility module data_load_utils
import data_load_utils as util
import test_data_load_utils as test

def convert_tweet_to_xy_generator1(tweet, length=160, window_size=40, step=3, batch_size=64):
    """ generator function that batch converts tweets (from pd DataFrame of tweets) to tuple of (x,y) 
    data, (where x is (m, window_size, character_set_size) ndarray and y is an (m,character_set_size) 
    dimensional array) suitable for feeding to keras fit_generator.
    Num training examples per tweet given by math.ceil((length - window_size)/step)"""

    assert length > window_size

    batch_num = 0
    n_batches = int(tweet.shape[0] / batch_size)  # terminate after last full batch for now

    # calculate num training examples per tweet
    m_per_tweet = int(ceil((length - window_size) / step))

    # get the universal character set and its index
    chars_univ, char_idx_univ = util.get_universal_chars_list()

    # allocate ndarray to contain one-hot encoded batch
    x_dims = (batch_size,             # num tweets
              m_per_tweet,
              window_size,
              len(chars_univ))        # length of the one-hot vector

    y_dims = (batch_size,             # num tweets
              m_per_tweet,
              len(chars_univ))        # length of the one-hot vector

    x_arr = np.zeros(shape=x_dims)
    y_arr = np.zeros(shape=y_dims)

    while batch_num < n_batches:  # in case tweet < batch_size

        # slice the batch
        this_batch = tweet.iloc[(batch_num*batch_size):(batch_num+1)*batch_size]

        # expand out all the tweets
        zipped = this_batch.apply(
            lambda x: util.get_series_data_from_tweet(
                x, length=length, window_size=window_size, step=step),
            axis=1)

        # unzips the tuples into separate tuples of x, y
        (x_tuple, y_tuple) = zip(*zipped)

        # turn each tuple into an series and then one-hot encode it
        x_bool = pd.Series(x_tuple).apply(lambda x: util.get_x_bool_array(x, chars_univ, char_idx_univ))
        y_bool = pd.Series(y_tuple).apply(lambda x: util.get_y_bool_array(x, char_idx_univ))

        # convert it to the ndarray
        for i, twit in enumerate(x_bool):
            x_arr[i] = twit

        for i, nchar in enumerate(y_bool):
            y_arr[i] = nchar

        # finally, reshape into a (m, w, c) array
        # where m is training example, w is window size,
        # c is one-hot encoded character
        x_fin = x_arr.reshape(batch_size * m_per_tweet, window_size, len(chars_univ))

        # y is a (m, c) array, where m is training example and c is one-hot encoded character
        y_fin = y_arr.reshape(batch_size * m_per_tweet, len(chars_univ))

        batch_num += 1  # do the next batch
        batch_num = batch_num % n_batches  # loop indefinitely

        yield (x_fin, y_fin)


In [33]:
import numpy as np
import pandas as pd
from math import ceil

Let's create some toy data to test it with

In [34]:
my_dict = {'text':
           ["red and yellow and pink and green, orange and purple and blue, I can sing a rainbow, sing a rainbow, sing a rainbow too",
            "sweet dreams are made of this, who am I to disagree, travel the world and the even seas, every body's looking for someone"],
           'emoji':
           [":rainbow:",
            ":gay_pride_flag:"]}

truncate_length = 160
window_size = 64
step = 3
chars, _ = util.get_universal_chars_list()


my_data = pd.DataFrame(my_dict)
x, y = util.convert_tweet_to_xy(my_data,
                                length=truncate_length,
                                window_size=window_size,
                                step=step)


print ("x shape:", x.shape[0], x.shape[1], x.shape[2])
print ("y shape:", y.shape[0], y.shape[1])


x shape: 64 64 93
y shape: 64 93


In [35]:
my_generator = convert_tweet_to_xy_generator1(my_data, length=truncate_length, \
                                                         window_size=window_size,step=step, batch_size=1)

def run_gen():
    for i in range(1):
        x1, y1 = next(my_generator)
        
run_gen()

# Memory profile `convert_tweet_to_xy()` vs the generator implementation 

In [9]:
%load_ext line_profiler
%load_ext memory_profiler

In [10]:
# First let's try the original implementation (ie hold everything in memory)
# %memit x, y = util.convert_tweet_to_xy(my_data, length=truncate_length, window_size=window_size,step=step)
%timeit x, y = util.convert_tweet_to_xy(my_data, length=truncate_length, window_size=window_size,step=step)

3.55 ms ± 158 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
# Now let's try the generator version
# %memit my_generator = util.convert_tweet_to_xy_generator(my_data, length=truncate_length, \
#                                                          window_size=window_size,step=step, batch_size=1)

%timeit my_generator = util.convert_tweet_to_xy_generator(my_data, length=truncate_length, \
                                                         window_size=window_size,step=step, batch_size=1)

547 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [ ]:
#%memit (x1, y1) = next(my_generator)

In [ ]:
def run_gen():
    for i in range(1):
        x1, y1 = next(my_generator)
        
%memit run_gen()

As expected, we can see that the generator version is significantly less RAM intensive than the non-generator version, though it's a bit hard to tell exactly, due to how RAM is managed inside a Jupyter notebook. Executing each cell independently (ie restarting the kernel between running each one) it looks like executing the original version increments RAM by 0.66 MB, while the generator version takes 0.16MB to create the generator object and another 0.28MB to generate both instances of the batch.

Next, let's scale things up and memory profile both functions on the tweets dataset. Comment/uncomment the various lines below to memory/CPU profile.

In [12]:
%memit tweets = util.filter_tweets_min_count(util.read_tweet_data('data/emojis_homemade.csv'), min_count=1000)

%memit tweets['text'] = util.filter_text_for_handles(tweets['text'])

/home/nickdbn/anaconda3/envs/deeplearning/lib/python3.6/site-packages/memory_profiler.py:336: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  returned = f(*args, **kw)


peak memory: 245.61 MiB, increment: 154.21 MiB
peak memory: 282.75 MiB, increment: 48.00 MiB


Let's use slightly different parameters to the previous version, let's use a window size of 64, which with a step of 3 and a total length of 160 gives 32 lines per tweet. As this is a power of 2, it'll mean we can generate batch sizes which have a number of training examples that's also a power of 2.

In [13]:
MAX_TWEET_LENGTH = 160
WINDOW_SIZE = 64
STEP = 3

chars_univ, chars_univ_idx = util.get_universal_chars_list()

In [14]:
%memit tweets_train = tweets.iloc[0:2048] 

peak memory: 283.17 MiB, increment: 0.00 MiB


In [15]:
import time

In [ ]:
#tic = time.time()

# %memit train_x, train_y = util.convert_tweet_to_xy(tweets_train)
%timeit train_x, train_y = util.convert_tweet_to_xy(tweets_train)

#print ("completed in", time.time()-tic, "s")

Loading 2048 tweets, the original version increments RAM by 2289.9MB, with mean execution time of 1.47s (SD 40ms).

In [17]:
#%memit my_generator = util.convert_tweet_to_xy_generator(tweets_train, length=truncate_length, \
#                                                         window_size=window_size,step=step, batch_size=64)
my_generator = util.convert_tweet_to_xy_generator(tweets_train, length=truncate_length, \
                                                         window_size=window_size,step=step, batch_size=64)


#tic = time.time()
def run_gen_tweets():
    for i in range(32): # 2048 (tweets) / 64 (batch size)
        train_x1, train_y1 = next(my_generator)


#%memit run_gen_tweets()
%timeit run_gen_tweets()

#print ("completed in", time.time()-tic, "s")

1.39 s ± 13.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Implemented as 32 batches of 64 tweets, the generator function took incremented the RAM by 93MB, with a peak of 377. In terms of CPU time, it was actually slightly faster, taking a mean 1.39s to execute the `run_get_tweets()` loop (SD 13ms) - the line that assigns the generator ran in nanoseconds.

# Generator Function Summary

Overall, implementing the data munging part of the model (taking a dataframe and turning it into a ndarray of the right sort of shape for feeding to a keras model) as a generator function, rather than trying to hold it all in RAM, was effective in keeping the RAM footprint of the data low, which should remove the RAM limitation on the size of the training set. It was surprising to me that this actually appeared to be faster as well.

Next, to put it into production, and train the LSTM on a massive data set!